This chapter's examples will be conducted with a new Vector class, which can takes on an arbitraliry number of dimensions.

Look at the comments in the code for detailed explanation of changes.

In [6]:
import math
from array import array
import reprlib
import numbers
import functools
import operator

class Vector:
    
    typecode = 'd'
    
    # the constructor takes input as a sequence instead of x, y
    def __init__(self, components):
        self._components = array(self.typecode, components)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # return memv without unpacking. This is because of the change in the constructor.
    
    def __iter__(self):
        # return an item created by iter()
        return (iter(self._components))
    
    def __repr__(self):
        # user reprlib.repr to make the resulting string truncated.
        class_name = type(self).__name__
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return '{}({})'.format(class_name, components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __abs__(self):
        # explicit computation of the length of vector.
        return math.sqrt(sum(x * x for x in self._components))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])
                + bytes(self.typecode, self._components))
    
    def __len__(self):
        return len(self._components)
    
    # this method makes the objects sliceable.
    # if inputted index is a number, return a value.
    # if inputted index is a slice, return a vector.
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, numbers.Integral):
            return self._components[index]
        elif isinstance(index, slice):
            return cls(self._components[index])
        else:
            error_msg = '{cls.__name__} indices must be integers.'
            raise TypeError(msg.format(cls=cls))
    
    # this method makes the objects able to be accessed
    # by convenient calls, e.g. vector.x, vector.y .
    short_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.short_names.find(name)
            if 0 <= pos and pos < len(cls.short_names):
                return self._components[pos]
        error_msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(error_msg.format(cls, name))
    
    # whenever __getattr__ is defined, __setattr__ should also
    # be defined to make things consistent.
    # this method helps prevent modifying vector's values.
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1 and name in cls.short_names:
            error = 'readonly attribute {attr_name!r}'
            msg = error.format(attr_name=name)
            raise AttributeError(msg)
        super().__setattr__(name, value)
    
    def __eq__(self, other):
        return all(a == b for a, b in zip(self, other))
    
    # this function, together with __eq__, makes Vector hashable
    def __hash__(self):
        if not hasattr(self, '_hash'):
            hashes = (hash(x) for x in self._components)
            self._hash = functools.reduce(operator.xor, hashes)
        return self._hash
            

In [8]:
v = Vector((2, 3, 4))
print('v[2]:', v[2])
print('hash(v):', hash(v))
print('v == [2, 3, 4] is', v == [2, 3, 4])

v[2]: 4.0
hash(v): 5
v == [2, 3, 4] is True


### Protocols and Duck Typing

Duck Typing means as long as an object behaves like a duck in all aspects that we need a duck to behave, we consider it a duck without knowing if it is really a duct or not.

A protocol is an informal interface, defined only in documentation and not in code.
<br>
For example, a protocol of a sequence is to have \_\_len\_\_ and \_\_getitem\_\_ implemented.

### Dynamic attribute access

Generally, when an object's attribute is called, if Python cannot found that object attribute, it find that attribute of the class, if still not found, it find that attribute of the super class, if still not found, it queries the \_\_getattr\_\_ method.

Look at the code above to see the implementation of attribute getter and setter for this Vector class.

### Hashing and a faster == 

Note that zip and map functions are all lazy (i.e. they produce generator).

The old \_\_eq\_\_ is slow because every time it is called, it create 2 tuples of the values.
<br>
Thus, we change it to the new version to make it more efficient.

itertools.chain() takes as input some sequences and output a generator of those input combined.